<a href="https://colab.research.google.com/github/pietrodileo/Python_for_MD_thesis/blob/main/TrainCNN_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🚀 Install, Import, Login

## Import Libraries

In [83]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import os
import wave
import pylab
import sys
from pathlib import Path
from scipy import signal
from scipy.io import wavfile
from sklearn.metrics import confusion_matrix
from keras.utils.vis_utils import plot_model
import itertools
import time
import shutil
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
!pip install pyyaml h5py  # Required to save models in HDF5 format

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [84]:
!pip install -U kaleido
#need to restart runtime after install kaleido
import kaleido
import plotly.express as px

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [85]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 📈 Preparing the data
We can now load the spectrograms into memory. We use the image_dataset_from_directory utility to generate the datasets. The validation set is what will ultimately be our benchmark when becomes to performance and accuracy of our classifier. The seed is for reproducibility.

##Create dataset functions

In [86]:
def createTrain_and_Test_Dataset(directory,BATCH_SIZE,VAL_SPLIT,IMAGE_HEIGHT, IMAGE_WIDTH, MODE):
  # Make a dataset containing the training spectrograms
  train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
                                            directory,
                                            labels='inferred',
                                            class_names = False,
                                            batch_size=BATCH_SIZE,
                                            validation_split=VAL_SPLIT,
                                            subset='training',
                                            shuffle=True,
                                            color_mode= MODE,
                                            image_size=(IMAGE_HEIGHT, IMAGE_WIDTH),
                                            seed=0)

  # Make a dataset containing the validation spectrogram
  valid_dataset = tf.keras.preprocessing.image_dataset_from_directory(
                                            directory,
                                            labels='inferred',
                                            class_names = False,
                                            batch_size=BATCH_SIZE,
                                            validation_split=VAL_SPLIT,
                                            subset='validation',
                                            shuffle=True,
                                            color_mode= MODE,
                                            image_size=(IMAGE_HEIGHT, IMAGE_WIDTH),
                                            seed=0)
  return train_dataset, valid_dataset

Before we can build our model and start training, we need to apply one simple augmentation the dataset and that is rescaling. We convert input from int to float32 and rescale it from the (0, 255) range to the (0,1) range.

## Rescale data size as [256, 256]

In [87]:
# Function to prepare our datasets for modelling
def prepare(batches):

  def normalize(img, label):
    return img / 255.0, label

  ds = (batches
        .map(normalize)
        .prefetch(tf.data.AUTOTUNE)
        ) 
  return ds

# ⏰ Execution time

In [88]:
start_time = time.time()

# 🧠 Define the Model and Train

## Define Model Architecture 🦾

In [89]:
def make_model(IMAGE_HEIGHT, IMAGE_WIDTH, N_CHANNELS, N_CLASSES, num_cnn_layers):
  # Create CNN model with 3 Convolution Layer Architecture
  model = tf.keras.models.Sequential()
  model.add(tf.keras.layers.Input(shape=(IMAGE_HEIGHT, IMAGE_WIDTH, N_CHANNELS)))
  # Conv2D(NumFilter, FilterSize, option...)
  # # 1
  # model.add(tf.keras.layers.Conv2D(32, (3, 3), strides=(1, 1), padding='same', activation='relu')) #kernel_initializer='he_uniform'
  # model.add(tf.keras.layers.BatchNormalization())
  # model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
  # model.add(tf.keras.layers.BatchNormalization())
  # # 2
  # model.add(tf.keras.layers.Conv2D(64, (3, 3),strides=(1, 1), padding='same', activation='relu'))
  # model.add(tf.keras.layers.BatchNormalization())
  # model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
  # model.add(tf.keras.layers.BatchNormalization())
  # # # 3
  # model.add(tf.keras.layers.Conv2D(128, (3, 3),strides=(1, 1), padding='same', activation='relu'))
  # model.add(tf.keras.layers.BatchNormalization())
  # model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
  # model.add(tf.keras.layers.BatchNormalization())
  # # # 4
  # model.add(tf.keras.layers.Conv2D(256, (3, 3), strides=(1, 1), padding='same', activation='relu'))
  # model.add(tf.keras.layers.BatchNormalization())
  # model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
  # model.add(tf.keras.layers.BatchNormalization())

  NUM_FILTERS = 32
  for i in range(1, num_cnn_layers+1):
    model.add(tf.keras.layers.Conv2D(NUM_FILTERS*i, (3,3), strides=(1, 1), activation='relu', padding='same'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(tf.keras.layers.BatchNormalization())

  # flattening
  model.add(tf.keras.layers.Flatten())
  # NN 
  model.add(tf.keras.layers.Dense(256, activation='relu')) 
  model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.Dropout(0.5))
  model.add(tf.keras.layers.Dense(N_CLASSES, activation='softmax'))
  return model

## Plot results function

In [90]:
def plotResults(history,SaveFile,outputPath):
  # Plot the loss curves for training and validation.
  history_dict = history.history
  loss_values = history_dict['loss']
  val_loss_values = history_dict['val_loss']
  epochs = range(1, len(loss_values)+1)
  
  d = {'Train Loss': loss_values, 'Test Loss': val_loss_values, 'Epochs':epochs}
  df = pd.DataFrame(d)
  
  fig = go.Figure()
  fig = make_subplots(rows=2, cols=1,
                      subplot_titles=("Training and Validation Loss", "Training and Validation Accuracy"))

  fig.append_trace(go.Scatter(
      x=df['Epochs'],
      y=df['Train Loss'],
      name="Training Loss",       # this sets its legend entry
      mode='lines+markers'
  ),1,1),

  fig.append_trace(go.Scatter(
      x=df['Epochs'],
      y=df['Test Loss'],
      name="Validation Loss",
      mode='lines+markers'
  ),1,1),

  # Plot the accuracy curves for training and validation.
  acc_values = history_dict['accuracy']
  val_acc_values = history_dict['val_accuracy']
  epochs = range(1, len(acc_values)+1)

  d = {'Train Accuracy': acc_values, 'Test Accuracy': val_acc_values, 'Epochs':epochs}
  df = pd.DataFrame(d)

  fig.append_trace(go.Scatter(
      x=df['Epochs'],
      y=df['Train Accuracy'],
      name="Training Accuracy",       # this sets its legend entry
      mode='lines+markers'
  ),2,1),

  fig.append_trace(go.Scatter(
      x=df['Epochs'],
      y=df['Test Accuracy'],
      name="Validation Accuracy",
      mode='lines+markers'
  ),2,1),

  # edit axis labels
  fig['layout']['xaxis']['title']='Epochs'
  fig['layout']['xaxis2']['title']='Epochs'
  fig['layout']['yaxis']['title']='Loss'
  fig['layout']['yaxis2']['title']='Accuracy'

  # Tick Distance
  fig['layout']['xaxis']['dtick']= 5
  fig['layout']['xaxis2']['dtick']= 5
  # First value on x axis
  fig['layout']['xaxis']['tick0']= 0
  fig['layout']['xaxis2']['tick0']= 0
  # Tick Mode
  fig['layout']['xaxis']['tickmode']= 'linear'
  fig['layout']['xaxis2']['tickmode']= 'linear'

  fig.update_layout(
      height=800, 
      width=1200,
      title="Loss and Accuracy",
      legend_title="Dataset",
      font=dict(size=14)
  )

  return fig, loss_values, val_loss_values, acc_values, val_acc_values

# 3. 🛫 Compute Code

## Define Parameters for Training

In [91]:
#@title Declare Constants
#@markdown ---
TASK = '100EP_2conv_Batch32_adam' #@param {type: "string"}
num_cnn_layers = 2 #@param {type: "number"}
IMAGE_HEIGHT = 256 #@param {type: "number"}
IMAGE_WIDTH = 256 #@param {type: "number"}
BATCH_SIZE = 32 #@param {type: "slider", min: 1, max: 256}
EPOCHS = 100  #@param {type: "number"}
VAL_SPLIT = 0.2  #@param {type: "slider", min: 0, max: 1, step:0.05}
MODE = "grayscale"  #@param ['rgb', 'rgba', 'grayscale']
SaveFile = True #@param {type: "boolean"}
#@markdown ---

##Select Input and Output ⏏

Speed up Google Colab by copying the zip file containing the images directly on the local path (75% faster). 

In [92]:
fileName = 'C1'
zipfile = fileName + '.zip'
directory = '/content/drive/MyDrive/TesiMagistrale/outputSpectrogram/CNN_TrainingIMGs/BW/'

zipPath = os.path.join(directory,zipfile)
outputDir = os.path.join(directory,'Risultati')
outputPath = os.path.join(outputDir,TASK)

if not os.path.exists(outputPath):
   # Create a new directory because it does not exist
   os.makedirs(outputPath)
   print("The output folder has been created!")

# Location of Zip File
drive_path = zipPath
local_path = '/content'

zipCopyPath = os.path.join(local_path,zipfile)
if not os.path.exists(zipCopyPath):
  # Copy the zip file and move it up one level (AKA out of the drive folder)
  !cp '{drive_path}' .
else:
  print('Files already transferred from Drive')

if not os.path.exists(fileName):
  # Navigate to the copied file and unzip it quietly
  os.chdir(local_path)
  !unzip -q '{zipfile}'
else:
  print('Files already unzipped')
# change directory to the new one
NewDir = os.path.join(local_path,fileName)

Files already transferred from Drive
Files already unzipped


##Select channels and classes, split the dataset

In [93]:
# Select number of channels
if MODE == 'grayscale':
  N_CHANNELS = 1
elif MODE == 'rgb':
  N_CHANNELS = 3
elif MODE == 'rgba':
  N_CHANNELS = 4

# Create the dataset and select number of classes
[train_batches, valid_batches] = createTrain_and_Test_Dataset(NewDir,BATCH_SIZE,VAL_SPLIT,IMAGE_HEIGHT, IMAGE_WIDTH, MODE)

classNames = train_batches.class_names
N_CLASSES = len(classNames)

# Extract image path for training and validation set
image_paths_train = train_batches.file_paths
image_paths_valid = valid_batches.file_paths

train_dataset = prepare(train_batches)
valid_dataset = prepare(valid_batches)

LabelNames = valid_batches.class_names
LabelNames

Found 1192 files belonging to 2 classes.
Using 954 files for training.
Found 1192 files belonging to 2 classes.
Using 238 files for validation.


['class_HealthyControl', 'class_advanced_PD-OFF']

## Set Optimizer and Loss Function ♟
### LOSS FUNCTION
see: 
* tf.keras.losses

### OPTIMIZERS 
See: 
* tf.keras.optimizers.RMSprop()
* tf.keras.optimizers.SGD
* tf.keras.optimizers.Adam

We introduce **Early Stopping**, which has the following parameters:
* min_delta: Minimum change in the monitored quantity to qualify as an improvement, i.e. an absolute change of less than min_delta, will count as no improvement.
* patience: Number of epochs with no improvement after which training will be stopped
* mode: One of {"auto", "min", "max"}. In min mode, training will stop when the quantity monitored has stopped decreasing; in "max" mode it will stop when the quantity monitored has stopped increasing; in "auto" mode, the direction is automatically inferred from the name of the monitored quantity.
* baseline: Baseline value for the monitored quantity. Training will stop if the model doesn't show improvement over the baseline.
* restore_best_weights: Whether to restore model weights from the epoch with the best value of the monitored quantity. If False, the model weights obtained at the last step of training are used. An epoch will be restored regardless of the performance relative to the baseline. If no epoch improves on baseline, training will run for patience epochs and restore weights from the best epoch in that set.

We also introduce **ModelCheckpoint**, which save in a specified path the model that achieved the best results and **ReduceLROnPlateau**, which reduce learning rate when a metric has stopped improving.




In [94]:
monitorValue = 'val_loss'
monitorMode = 'min'

checkpoint_path = os.path.join(outputPath,'training_1/cp.ckpt')
bestModelName = os.path.join(outputPath,'best_mdl.h5')
earlyStopping = EarlyStopping(monitor=monitorValue, 
                              patience=40, 
                              verbose=0, 
                              mode=monitorMode,
                              restore_best_weights=True)

mcp_save = ModelCheckpoint(filepath=checkpoint_path,
                           save_best_only=True, 
                           monitor=monitorValue, 
                           mode=monitorMode)

reduce_lr_loss = ReduceLROnPlateau(monitor=monitorValue,
                                   factor=0.1, patience=5, 
                                   verbose=0, min_delta=1e-4, mode=monitorMode)

LOSS = 'sparse_categorical_crossentropy'

# OPTIMIZER = tf.keras.optimizers.SGD(
#         learning_rate=0.01,
#         momentum=0.0,
#         nesterov=False,
#         name='SGD')

OPTIMIZER = tf.keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999)

/usr/local/lib/python3.7/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning:

The `lr` argument is deprecated, use `learning_rate` instead.



## Train the network 💪


### Make Model

In [95]:
model = make_model(IMAGE_HEIGHT, IMAGE_WIDTH, N_CHANNELS, N_CLASSES, num_cnn_layers)
# Compile model
model.compile(
    loss = LOSS,
    optimizer = OPTIMIZER,
    metrics=['accuracy'])

### Train Model

In [ ]:
# Train model and capture the history
history = model.fit(train_dataset, 
                    epochs=EPOCHS, 
                    validation_data = valid_dataset,
                    callbacks = [earlyStopping, mcp_save, reduce_lr_loss]
                    )

Epoch 1/100
30/30 [==============================] - ETA: 0s - loss: 1.0174 - accuracy: 0.6499

30/30 [==============================] - 31s 847ms/step - loss: 1.0174 - accuracy: 0.6499 - val_loss: 0.9977 - val_accuracy: 0.5798 - lr: 0.0010
Epoch 2/100
30/30 [==============================] - ETA: 0s - loss: 0.4211 - accuracy: 0.8145

30/30 [==============================] - 31s 882ms/step - loss: 0.4211 - accuracy: 0.8145 - val_loss: 0.8917 - val_accuracy: 0.5798 - lr: 0.0010
Epoch 3/100
30/30 [==============================] - 25s 609ms/step - loss: 0.2792 - accuracy: 0.9036 - val_loss: 1.3021 - val_accuracy: 0.5798 - lr: 0.0010
Epoch 4/100
30/30 [==============================] - 23s 610ms/step - loss: 0.1628 - accuracy: 0.9403 - val_loss: 1.0676 - val_accuracy: 0.5798 - lr: 0.0010
Epoch 5/100
30/30 [==============================] - 23s 613ms/step - loss: 0.1042 - accuracy: 0.9612 - val_loss: 1.0261 - val_accuracy: 0.5798 - lr: 0.0010
Epoch 6/100
30/30 [==============================] - ETA: 0s - loss: 0.0817 - accuracy: 0.9748

30/30 [==============================] - 29s 830ms/step - loss: 0.0817 - accuracy: 0.9748 - val_loss: 0.7575 - val_accuracy: 0.5798 - lr: 0.0010
Epoch 7/100
30/30 [==============================] - 26s 733ms/step - loss: 0.0448 - accuracy: 0.9853 - val_loss: 0.9849 - val_accuracy: 0.5798 - lr: 0.0010
Epoch 8/100
30/30 [==============================] - 23s 634ms/step - loss: 0.0202 - accuracy: 0.9969 - val_loss: 1.1266 - val_accuracy: 0.5798 - lr: 0.0010
Epoch 9/100
30/30 [==============================] - ETA: 0s - loss: 0.0248 - accuracy: 0.9937

30/30 [==============================] - 29s 837ms/step - loss: 0.0248 - accuracy: 0.9937 - val_loss: 0.6283 - val_accuracy: 0.5924 - lr: 0.0010
Epoch 10/100
30/30 [==============================] - ETA: 0s - loss: 0.0235 - accuracy: 0.9937

30/30 [==============================] - 30s 876ms/step - loss: 0.0235 - accuracy: 0.9937 - val_loss: 0.6127 - val_accuracy: 0.6008 - lr: 0.0010
Epoch 11/100
30/30 [==============================] - 26s 682ms/step - loss: 0.0168 - accuracy: 0.9979 - val_loss: 1.2711 - val_accuracy: 0.5798 - lr: 0.0010


## Plot results 🛬 

In [ ]:
fig, loss_values, val_loss_values, acc_values, val_acc_values = plotResults(history,SaveFile,outputPath)
fig.show()

## Save plot

In [ ]:
# Save pic in HTML
if SaveFile == True:
  AccLossPngOutput = os.path.join(outputPath,'results.html')
  fig.write_html(AccLossPngOutput)
  print('Image Saved')

In [ ]:
# Convert HTML in PNG and export to Google Drive
resultImg = f"{outputPath}"+'/results.png'
fig.to_image(format="png", engine="kaleido");
fig.write_image(resultImg)

##Export Results to Google Drive 💯

Save the best model

In [ ]:
model.save(bestModelName)

Import a previous model

In [ ]:
model = tf.keras.models.load_model(bestModelName)

Compute the final loss and accuracy

In [ ]:
final_loss, final_acc = model.evaluate(valid_dataset, verbose=0)
print("Final loss: {0:.6f}, final accuracy: {1:.6f}".format(final_loss, final_acc))

Compute runtime

In [ ]:
runtime = time.time() - start_time

print('Runtime:')
print("--- %s seconds ---" % (runtime))
print("--- %s minutes ---" % ((runtime)/60))

In [ ]:
# dictionary of lists  
InfoDict = {'IMAGE_HEIGHT':[IMAGE_HEIGHT], 'IMAGE_WIDTH':[IMAGE_WIDTH],
              'BATCH_SIZE': [BATCH_SIZE], 'N_CHANNELS': [N_CHANNELS], 'N_CLASSES': [N_CLASSES],
              'EPOCHS':[EPOCHS], 'VAL_SPLIT': [VAL_SPLIT], 'MODE': MODE,
              'FINAL VALIDATION LOSS': [final_loss], 'FINAL VALIDATION ACC': [final_acc],
              'RUNTIME (s)': [runtime], 'RUNTIME (min)': [runtime/60]
              }
InfoDict
dfInfo = pd.DataFrame(InfoDict)
dfInfo.index = ['CNN']
# saving the dataframe 
outputName = 'Model_Info_and_Performance.xlsx'
OutputFileName = os.path.join(outputPath,outputName)
if SaveFile == True:
  # writing to Excel
  dfInfo.to_excel(OutputFileName)

In [ ]:
# dictionary of lists  
OutputDict = {'Train_ACC':acc_values, 'Train_LOSS':loss_values, 
              'Test_ACC': val_acc_values, 'Test_LOSS': val_loss_values}
dfResult = pd.DataFrame(OutputDict) 
# saving the dataframe 
outputName = 'Risultati'
OutputFileName = outputPath+'/'+outputName+'.xlsx'
dfResult.to_excel(OutputFileName) 

In [ ]:
Summary = os.path.join(outputPath,'modelsummary.txt')
modelInfo = model.summary()

stringlist = []
model.summary(print_fn=lambda x: stringlist.append(x))
short_model_summary = "\n".join(stringlist)
print(short_model_summary)

with open(Summary, 'w') as f:
  # Pass the file handle in as a lambda function to make it callable
  f.write(short_model_summary)

In [ ]:
pngOutput = os.path.join(outputPath,'model_plot.png')
plot_model(model, to_file=pngOutput, show_shapes=True, show_layer_names=False)

## Confusion Matrix

In [ ]:
def Evaluate_Batches(batch_dataset):
  #extract the labels of each batch
  true_labels = []
  pred_labels = [] 
  train_labels = []
  for image_batch, label_batch in batch_dataset:
    # image_batch contains n figure, where n is batch size.  
    # It's an EagerTensor with a shape of (n, ImHeight, ImWidth, Nchannels)
    preds = model.predict(image_batch,verbose=0)
    # preds is a ndarray with n rows and N_CLASSES columns
    # it is the output of the last neurons of the Dens/softmax Layer
    pred_labels += np.argmax(preds, axis = - 1).tolist()
    train_labels += label_batch.numpy().tolist()
    # Alternative Code (use EagerTensor, a bit longer) 
    # pred_labels.append(np.argmax(preds, axis = - 1))
    # pred_labels is a list of which each element is an ndarray with size n (batchsize)
    # This list is composed by M elements, where M is the number of batch 
  return train_labels, pred_labels

def plot_confusion_matrix(cm,
                          target_names,
                          title='Confusion matrix',
                          cmap=None,
                          normalize=True):
    """
    given a sklearn confusion matrix (cm), make a nice plot

    Arguments
    ---------
    cm:           confusion matrix from sklearn.metrics.confusion_matrix

    target_names: given classification classes such as [0, 1, 2]
                  the class names, for example: ['high', 'medium', 'low']

    title:        the text to display at the top of the matrix

    cmap:         the gradient of the values displayed from matplotlib.pyplot.cm
                  see http://matplotlib.org/examples/color/colormaps_reference.html
                  plt.get_cmap('jet') or plt.cm.Blues

    normalize:    If False, plot the raw numbers
                  If True, plot the proportions

    Usage
    -----
    plot_confusion_matrix(cm           = cm,                  # confusion matrix created by
                                                              # sklearn.metrics.confusion_matrix
                          normalize    = True,                # show proportions
                          target_names = y_labels_vals,       # list of names of the classes
                          title        = best_estimator_name) # title of graph

    Citiation
    ---------
    http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html

    """
    import matplotlib.pyplot as plt
    import numpy as np
    import itertools

    accuracy = np.trace(cm) / np.sum(cm).astype('float')
    misclass = 1 - accuracy

    if cmap is None:
        cmap = plt.get_cmap('Blues')

    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title, fontsize=18)
    plt.colorbar()

    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=45, fontsize=13)
        plt.yticks(tick_marks, target_names, fontsize=13)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]


    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.4f}".format(cm[i, j]),
                     size='xx-large',
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]), 
                     size='xx-large',
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")


    plt.tight_layout()
    plt.ylabel('True label',fontsize=14)
    plt.xlabel('Predicted label\naccuracy={:0.4f}; misclass={:0.4f}'.format(accuracy, misclass),fontsize=14)
    plt.show()    

def ConfMtxPlot(train_labels, pred_labels):
  confusion_mtx = confusion_matrix(train_labels, pred_labels) 
  # plot the confusion matrix
  f,ax = plt.subplots(figsize=(8, 7))
  sns.heatmap(confusion_mtx, annot=True, linewidths=0.1,cmap="Blues",linecolor="k", fmt= '.0f',ax=ax,cbar=False)
  plt.xlabel("Predicted Label")
  plt.ylabel("True Label")
  plt.title("Confusion Matrix")
  plt.show()

In [ ]:
pred_labels, true_labels = Evaluate_Batches(valid_dataset)

In [ ]:
confusion_mtx = confusion_matrix(true_labels, pred_labels)
plot_confusion_matrix(confusion_mtx,
                          LabelNames,
                          title='Confusion matrix',
                          cmap=None,
                          normalize=False)

## Plot the loss and accuracy curves for training and validation 


In [ ]:
plt.plot(history.history['loss'], color='g', label="Training Loss")
plt.plot(history.history['val_loss'], color='b', label="Validation Loss")
plt.title("Training vs Validation Loss")
plt.xlabel("Number of Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(true_labels, pred_labels, target_names = LabelNames))

ROC Curve

In [ ]:
from sklearn.metrics import roc_curve, auc

FPR, TPR, ROC_Thresholds = roc_curve(true_labels, pred_labels)
AUC = auc(FPR, TPR)

plt.figure()
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(FPR, TPR, label='AUC = {:.3f}'.format(AUC))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='best')
plt.show()

Statistics for Binary classifications

In [ ]:
PositiveClass = 1 # select positive class (disease), 0 or 1, in binary classification

if len(LabelNames) == 2:
  if PositiveClass == 0:
    TP = confusion_mtx[0,0]
    FP = confusion_mtx[0,1]
    FN = confusion_mtx[1,0]
    TN = confusion_mtx[1,1]
  else:
    TN = confusion_mtx[0,0]
    FN = confusion_mtx[0,1]
    FP = confusion_mtx[1,0]
    TP = confusion_mtx[1,1]
  
  Accuracy = (TP+TN)/(TP+TN+FP+FN);
  Miscl_Rate = 1-Accuracy;
  PPV = TP/(TP+FP); # also called Precision
  NPV = TN/(TN+FN);
  Sensitivity = TP/(TP+FN); # Or Recall, True Positive Rate
  Specificity = TN/(TN+FP); # True Negative Rate
  J = Sensitivity+Specificity-1;
  F1score = 2*((PPV*Sensitivity)/(PPV + Sensitivity));
  Gmean = np.sqrt(Sensitivity*Specificity);

  true_labels = np.array(true_labels)
  pred_labels = np.array(pred_labels)
  errors = true_labels != pred_labels
  errorsNum = sum(errors)

  StatLabels = ["Accuracy","Misclassification Rate","PPV","NPV","Sensitivity",
                "Specificity","AUC","Youden's Index",
                "F1-score","G-Mean","Runtime (s)"]

  StatArray = np.array([Accuracy,Miscl_Rate,PPV,NPV,Sensitivity,Specificity,
               AUC,J,F1score,Gmean, runtime])

  Stat_df = pd.DataFrame(StatArray)#,columns=StatLabels)
  Stat_df = pd.DataFrame.transpose(Stat_df)
  Stat_df.columns = StatLabels
  Stat_df.index = ['Statistics']
Stat_df
# # Standard Error
# q1 = AUC/(2-AUC);
# q2 = 2*(AUC)^2/(1+AUC);
# SE = sqrt((AUC*(1-AUC)+(na-1)*(q1-AUC^2)+(nn-1)*(q2-AUC^2))/(na*nn));